In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
from scipy.io import mmread
import numpy as np
import pandas as pd
import seaborn as sns
#import fbpca
import csv
import sys
from sklearn import linear_model
from sklearn import decomposition
from sklearn import manifold
import glob
import itertools

csv.field_size_limit(sys.maxsize)
csv.field_size_limit(100000000)

9223372036854775807

##  Explore GSM2396856_dc_3hr

In [16]:

#------------------------------------------------------------------------#
# Read Data
## Matrix
mat=mmread(os.path.join(Data_dir, "GSM2396856_dc_3hr.mtx.txt"))
genes_path = os.path.join(Data_dir, "GSM2396856_dc_3hr_genenames.csv")
gene_names = pd.read_table(genes_path, sep=',', skiprows=0)
gene_names = gene_names.iloc[:,1]
barcodes_path = os.path.join(Data_dir, "GSM2396856_dc_3hr_cellnames.csv")
barcodes = pd.read_table(barcodes_path, sep=',', skiprows=0)
barcodes = list(barcodes.iloc[:,1])

## Get the GUIDE part of the X
cbc_gbc_dict_path = os.path.join(Data_dir, "GSM2396856_dc_3hr_cbc_gbc_dict_lenient.csv")
gbcs = [row[0] for row in csv.reader(open(cbc_gbc_dict_path))]
cbcs_raw = [row[1] for row in csv.reader(open(cbc_gbc_dict_path))]
cbcs = []
for temp_val in cbcs_raw:
    temp = temp_val.replace(' ','').split(',')
    cbcs.append(list(set(temp)&set(barcodes)))
gbc_cbc_dict = dict(zip(gbcs, cbcs))
X_guides = dict2X(GUIDES_DICT=gbc_cbc_dict, cbcs=barcodes)
#------------------------------------------------------------------------#




/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead.
  import sys
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: read_table is deprecated, use read_csv instead.
  # Remove the CWD from sys.path while we load stuff.


## Explore all families

## Gene names and barcodes

In [27]:
len(gene_names)

17775

In [33]:
len(barcodes)
barcodes

['AAACATACACGTAC_dc3hLPS_A8',
 'AAACATACATGTCG_dc3hLPS_A8',
 'AAACATACCAACTG_dc3hLPS_A8',
 'AAACATACTCCTTA_dc3hLPS_A8',
 'AAACATACTCTCCG_dc3hLPS_A8',
 'AAACATTGCTGTCC_dc3hLPS_A8',
 'AAACATTGCTTATC_dc3hLPS_A8',
 'AAACATTGGCGGAA_dc3hLPS_A8',
 'AAACATTGTACTGG_dc3hLPS_A8',
 'AAACCGTGCACCAA_dc3hLPS_A8',
 'AAACCGTGCCAACA_dc3hLPS_A8',
 'AAACCGTGTCATTC_dc3hLPS_A8',
 'AAACCGTGTGCCAA_dc3hLPS_A8',
 'AAACGCACGAGACG_dc3hLPS_A8',
 'AAACGCACTAACCG_dc3hLPS_A8',
 'AAACGCACTCGTGA_dc3hLPS_A8',
 'AAACGCACTGCTAG_dc3hLPS_A8',
 'AAACGCTGACGGTT_dc3hLPS_A8',
 'AAACGGCTTGGTAC_dc3hLPS_A8',
 'AAACTTGAAGCATC_dc3hLPS_A8',
 'AAACTTGAGGACAG_dc3hLPS_A8',
 'AAACTTGAGGTTAC_dc3hLPS_A8',
 'AAACTTGAGTTCTT_dc3hLPS_A8',
 'AAAGACGACCGCTT_dc3hLPS_A8',
 'AAAGACGAGAGGCA_dc3hLPS_A8',
 'AAAGACGATACAGC_dc3hLPS_A8',
 'AAAGACGATCAGGT_dc3hLPS_A8',
 'AAAGCAGAGTTTGG_dc3hLPS_A8',
 'AAAGCAGATTTGGG_dc3hLPS_A8',
 'AAAGCCTGAGGCGA_dc3hLPS_A8',
 'AAAGCCTGCGACTA_dc3hLPS_A8',
 'AAAGCCTGTCACGA_dc3hLPS_A8',
 'AAAGGCCTAAACAG_dc3hLPS_A8',
 'AAAGGCCT

In [34]:
#guide barcodes
len(gbcs)

324

In [33]:
#cell barcodes
len(cbcs)

1390

## List of all cell ids

In [14]:
#print(cbcs)

## Append all families together and then read,generate required perturb, cell lists, expression matrix etc 

### Specify paths

In [8]:
#mat=mmread(os.path.join(Data_dir, "GSM2396856_dc_3hr.mtx.txt"))
Data_dir = "/home/user/Downloads/GSE90063/GSE90063_RAW/"

matrix_paths = [file for file in glob.glob(Data_dir+'/*mtx.*')]
genes_paths = [file for file in glob.glob(Data_dir+'/*genenames*')] 
barcodes_paths = [file for file in glob.glob(Data_dir+'/*cellnames*')] 
cbc_gbc_dict_paths = [file for file in glob.glob(Data_dir+'/*cbc_gbc_dict*')] 

In [4]:
def read_matrix(mat):
    """
    -convert .mtx file to data frame
    -map to corresponding genenames and barcodes
    """
    
    DGE=pd.DataFrame(mat.toarray())
    DGE.index=gene_names
    DGE.columns=barcodes
    
    return DGE

def dict2X(GUIDES_DICT,cbcs):
    """convert guide cbc dictionary into covariate matrix"""
    X=pd.DataFrame()
    for key in GUIDES_DICT.keys():
        curkey=[]
        for cbc in cbcs:
            if cbc in GUIDES_DICT[key]:
                curkey.append(1)
            else:
                curkey.append(0)
        X[key]=np.array(curkey)
    X.index=cbcs
    return X

def get_barcodes_genenames(barcodes_paths,genes_paths):
    """create list of all barcodes and gene_names"""
    barcodes =[]
    gene_names =[]

    for barcodes_path,genes_path in itertools.zip_longest(barcodes_paths,genes_paths):
        barcodes1 = pd.read_table(barcodes_path, sep=',', skiprows=0)
        barcodes1 = list(barcodes1.iloc[:,1])
        barcodes.extend(barcodes1)
        gene_names1 = pd.read_table(genes_path, sep=',', skiprows=0)
        gene_names1 = gene_names1.iloc[:,1]
        gene_names.extend(gene_names1)
        
    return barcodes, gene_names

def get_gcdict_expmatrix(cbc_gbc_dict_paths):
    """retrieve guide cbc dictionary and generate expression matrix"""
    gbcs = []
    cbcs_raw = []
    cbcs = []
    for cbc_gbc_dict_path in cbc_gbc_dict_paths:
        gbcs.extend([row[0] for row in csv.reader(open(cbc_gbc_dict_path))])
        cbcs_raw.extend([row[1] for row in csv.reader(open(cbc_gbc_dict_path))])

        for temp_val in cbcs_raw:
            temp = temp_val.replace(' ','').split(',')
            cbcs.append(list(set(temp)&set(barcodes)))
    gbc_cbc_dict = dict(zip(gbcs, cbcs))
    ExpressionMatrix = dict2X(GUIDES_DICT=gbc_cbc_dict, cbcs=barcodes)
    
    return gbc_cbc_dict, ExpressionMatrix

def readDictionaryFile(barcodes, cbc_gbc_dict_paths):
    
    '''
    Function to read the dictionary file and map with corresponding perturbagents
    Returns:
    -list of perturbagents
    -dictionary which maps the cell with perturbation
    '''
    perturbationList=[]
    dictionary=dict()
    newCellList=[]
    for cbc_gbc_dict_path in cbc_gbc_dict_paths:
        with open(cbc_gbc_dict_path) as csv_file:
            csv_reader=csv.reader(csv_file,delimiter=',')
            total_line=list(csv_reader)
            
            for line in total_line:
                perturbationList.append(line[0])
                newline=line[1].split(',')
                for cell in newline:
                    if cell[0]==' ' and cell[1:len(cell)] in cellList:
                        if cell[1:len(cell)] in dictionary:
                            dictionary[cell[1:len(cell)]].append(line[0])
                        else:
                            dictionary[cell[1:len(cell)]]=[line[0]]
                    elif cell[0]!=' ' and cell[1:len(cell)] in cellList:
                        if cell in dictionary:
                            dictionary[cell].append(line[0])
                        else:
                            dictionary[cell]=[line[0]]
    return perturbationList, dictionary


## Perturbagents mapped to cell ids

In [9]:
gbc_cbc_dict, ExpressionMatrix = get_gcdict_expmatrix(cbc_gbc_dict_paths)
    

In [10]:
len(gbc_cbc_dict.keys())

120

In [31]:
#print(gbc_cbc_dict)

{'p_sgEGR1_3': ['GAGTACTGGGTCAT_dc3hLPS_C9',
  'TGGACCCTGTTCTT_dc3hLPS_B9',
  'ATCCATACCACACA_dc3hLPS_D9',
  'GAAGCTACTTGCGA_dc3hLPS_B9',
  'ATAGGAGAATCTTC_dc3hLPS_D8',
  'TACTCTGACGTTGA_dc3hLPS_C8',
  'TCACCGTGAAACGA_dc3hLPS_D9',
  'ACACGATGTACTCT_dc3hLPS_D9',
  'ACGATTCTCACTAG_dc3hLPS_C9',
  'TTGATCTGAAGAGT_dc3hLPS_D8',
  'CGAAGACTAGGTCT_dc3hLPS_D8',
  'ACGGGAGAGTAAGA_dc3hLPS_D8',
  'GAGGGATGGCTTCC_dc3hLPS_D8',
  'CGCGGATGCAAGCT_dc3hLPS_B8',
  'GAACACACGCGTTA_dc3hLPS_B8',
  'CACTGCTGGAGGCA_dc3hLPS_D8',
  'TGCAATCTAATCGC_dc3hLPS_C8',
  'CCCTCAGACTTAGG_dc3hLPS_C8',
  'ACCTGGCTGCTACA_dc3hLPS_C9',
  'TATAGCCTTGACTG_dc3hLPS_D8',
  'TAATCCACGCGATT_dc3hLPS_C8',
  'TTACTCGACTTGAG_dc3hLPS_D9',
  'TAATCGCTTCTTCA_dc3hLPS_C9',
  'TAATGTGACCCTAC_dc3hLPS_A9',
  'ATTGTAGAATTCGG_dc3hLPS_B8',
  'CGTCCATGTGTCCC_dc3hLPS_B8',
  'CACACCTGAAGATG_dc3hLPS_D8',
  'ACTCTCCTCAATCG_dc3hLPS_B9',
  'CCGAAAACAGTAGA_dc3hLPS_B8',
  'CCACCATGACACGT_dc3hLPS_A8',
  'GCTCACTGCCTGAA_dc3hLPS_A8',
  'GACGGCACCATGGT_dc3hLPS

## ExpressionMatrix

In [11]:
ExpressionMatrix.head()

,p_sgEGR1_3,p_sgEGR1_2,p_sgEGR1_4,p_sgCREB1_2,p_sgNR2C2_2,p_sgNR2C2_3,p_sgNR2C2_5,p_sgCREB1_4,p_sgCREB1_5,p_sgYY1_3,...,c_sgPTGER2_4,c_sgPTGER2_1,c_sgPTGER2_3,c_sgPTGER2_2,c_sgCIT_7,c_sgCIT_1,c_sgOGG1_4,c_sgOGG1_3,c_sgOGG1_2,m_Egr1_3
AAACATACACGTAC_dc3hLPS_A8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
AAACATACATGTCG_dc3hLPS_A8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACATACCAACTG_dc3hLPS_A8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACATACTCCTTA_dc3hLPS_A8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACATACTCTCCG_dc3hLPS_A8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
perturbList, cell_perturb = readDictionaryFile(barcodes)

## List of all Perturbagents

In [21]:
len(perturbList)

324

In [12]:
#print(perturbList)

## Cells mapped to the applied perturbagents

In [22]:
len(cell_perturb)

166643

In [13]:
#print(cell_perturb)